In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO

In [ ]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# Project structure
PROJECT_ROOT = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(PROJECT_ROOT, "data", "detection")

IMAGES_DIR = os.path.join(DATA_PATH, "images")
LABELS_DIR = os.path.join(DATA_PATH, "labels")

IMAGES_TRAIN_DIR = os.path.join(IMAGES_DIR, "train")
IMAGES_VAL_DIR = os.path.join(IMAGES_DIR, "val")

LABELS_TRAIN_DIR = os.path.join(LABELS_DIR, "train")
LABELS_VAL_DIR = os.path.join(LABELS_DIR, "val")

In [ ]:
for p in (IMAGES_TRAIN_DIR, IMAGES_VAL_DIR, LABELS_TRAIN_DIR, LABELS_VAL_DIR):
    if not os.path.exists(p):
        print("WARNING: path does not exist:", p)

In [ ]:
yaml_path = os.path.join(DATA_PATH, "face_detection.yaml")
with open(yaml_path, "w") as f:
    f.write(f"""
path: {DATA_PATH}
train: images/train
val: images/val
nc: 1
names: ['face']
""")
print("YAML file created at:", yaml_path)

In [ ]:
def visualize_random_samples(n=3):
    imgs = [f for f in os.listdir(IMAGES_TRAIN_DIR) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    if len(imgs) == 0:
        print("No training images found in", IMAGES_TRAIN_DIR)
        return
    for _ in range(n):
        fn = random.choice(imgs)
        img_path = os.path.join(IMAGES_TRAIN_DIR, fn)
        lbl_path = os.path.join(LABELS_TRAIN_DIR, fn.rsplit('.',1)[0] + '.txt')
        img = cv2.imread(img_path)
        if img is None:
            print("Failed to read:", img_path); continue
        h, w = img.shape[:2]
        if os.path.exists(lbl_path):
            with open(lbl_path, 'r') as lf:
                for line in lf:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                    cls, xc, yc, bw, bh = map(float, parts[:5])
                    x1 = int((xc - bw/2) * w)
                    y1 = int((yc - bh/2) * h)
                    x2 = int((xc + bw/2) * w)
                    y2 = int((yc + bh/2) * h)
                    cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
                    cv2.putText(img, "face", (max(0,x1), max(0,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(6,6)); plt.imshow(img_rgb); plt.axis('off'); plt.show()

In [ ]:
print("Showing a few training samples...")
visualize_random_samples(3)

In [ ]:
model = YOLO("yolo11n.pt")
print("Loaded model:", model)

In [ ]:
results = model.train(
    data=yaml_path,
    model="yolo11n.pt",
    epochs=10,
    imgsz=224,
    batch=4,
    workers=2,
    optimizer="AdamW",
    lr0=1e-3,
    patience=5,
    device=0,
    pretrained=True,
    project="runs/detect",
    name="face_yolo11n",

    # 👇 optional flags to limit storage
    save=True,             # ✅ only saves final model weights
    save_period=-1,        # ✅ disables saving intermediate epochs
    exist_ok=True,         # avoids duplicate folder creation
    cache=False,           # avoids dataset caching to disk
    plots=False,           # 🚫 disables result image saving
    verbose=False,         # less console clutter
)

In [ ]:
print("Training finished. run directory:", results.save_dir)

In [ ]:
metrics = model.val()  # runs validation and returns metrics dict
print("Validation metrics:", metrics)

In [ ]:
best_weights = os.path.join(results.save_dir, "weights", "best.pt")
print("Best weights expected at:", best_weights)
trained = YOLO(best_weights)

In [ ]:
val_images = [f for f in os.listdir(IMAGES_VAL_DIR) if f.lower().endswith(('.jpg','.jpeg','.png'))]
if len(val_images) == 0:
    print("No validation images found in", IMAGES_VAL_DIR)
else:
    # predict per-file (we will process sequentially to avoid heavy memory spikes)
    out_dir = os.path.join(results.save_dir, "predictions_on_val")
    os.makedirs(out_dir, exist_ok=True)
    for fname in val_images:
        src = os.path.join(IMAGES_VAL_DIR, fname)
        # save results for each image
        preds = trained.predict(source=src, imgsz=416, conf=0.5, save=True, project=out_dir, name=fname.rsplit('.',1)[0])
    print("Saved per-image predictions under:", out_dir)

In [ ]:
saved_dir = os.path.join(out_dir, "predictions_on_val")
if os.path.exists(saved_dir):
    saved_files = [f for f in os.listdir(saved_dir) if f.lower().endswith(('.jpg','.png'))]
    if len(saved_files) == 0:
        print("No saved prediction images found in", saved_dir)
    else:
        # show up to 3
        to_show = saved_files[:3]
        for fn in to_show:
            imgp = os.path.join(saved_dir, fn)
            img = cv2.imread(imgp)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(6,6)); plt.imshow(img); plt.axis('off'); plt.show()
else:
    print("Predictions folder not found:", saved_dir)